In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [256, 256]  # 单层LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 构建模型
class BiLSTM(object):
    """
    Bi-LSTM 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):

            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2]
                    self.embeddedWords = tf.concat(outputs, 2)
        
        # 去除最后时间步的输出作为全连接的输入
        finalOutput = self.embeddedWords[:, -1, :]
        
        outputSize = config.model.hiddenSizes[-1] * 2  # 因为是双向LSTM，最终的输出值是fw和bw的拼接，因此要乘以2
        output = tf.reshape(finalOutput, [-1, outputSize])  # reshape成全连接层的输入维度
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTM(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

        outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

        builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/sparsity is illegal; 

2019-01-17T20:44:38.594572, step: 45, loss: 0.6854075789451599, acc: 0.4688, auc: 0.5507, precision: 0.0, recall: 0.0
2019-01-17T20:44:39.245404, step: 46, loss: 0.7037606239318848, acc: 0.5391, auc: 0.4776, precision: 0.4, recall: 0.0345
2019-01-17T20:44:39.934466, step: 47, loss: 0.7124852538108826, acc: 0.4609, auc: 0.4049, precision: 0.25, recall: 0.0149
2019-01-17T20:44:40.598915, step: 48, loss: 0.6867026090621948, acc: 0.5391, auc: 0.5348, precision: 0.6667, recall: 0.0333
2019-01-17T20:44:41.329120, step: 49, loss: 0.7004528045654297, acc: 0.5078, auc: 0.4537, precision: 0.0, recall: 0.0
2019-01-17T20:44:42.058718, step: 50, loss: 0.6933881640434265, acc: 0.5078, auc: 0.5278, precision: 1.0, recall: 0.0156
2019-01-17T20:44:42.769413, step: 51, loss: 0.6953430771827698, acc: 0.4688, auc: 0.4995, precision: 0.0, recall: 0.0
2019-01-17T20:44:43.400523, step: 52, loss: 0.7019579410552979, acc: 0.5156, auc: 0.4475, precision: 0.0, recall: 0.0
2019-01-17T20:44:44.052988, step: 53, lo

2019-01-17T20:45:53.608265, step: 112, loss: 0.6500290036201477, acc: 0.5469, auc: 0.6883, precision: 1.0, recall: 0.0169
2019-01-17T20:45:54.345237, step: 113, loss: 0.6062589287757874, acc: 0.5859, auc: 0.7633, precision: 0.75, recall: 0.1053
2019-01-17T20:45:55.063181, step: 114, loss: 0.6920719146728516, acc: 0.5156, auc: 0.5779, precision: 0.5417, recall: 0.5735
2019-01-17T20:45:55.803638, step: 115, loss: 0.7255005836486816, acc: 0.5703, auc: 0.5718, precision: 0.4737, recall: 0.1667
2019-01-17T20:45:56.541664, step: 116, loss: 0.6941976547241211, acc: 0.5469, auc: 0.5643, precision: 0.7273, recall: 0.127
2019-01-17T20:45:57.278594, step: 117, loss: 0.6923136711120605, acc: 0.4531, auc: 0.4847, precision: 0.6667, recall: 0.0822
2019-01-17T20:45:58.013159, step: 118, loss: 0.6886915564537048, acc: 0.4922, auc: 0.4947, precision: 0.75, recall: 0.0448
2019-01-17T20:45:58.653627, step: 119, loss: 0.7095251679420471, acc: 0.5156, auc: 0.4384, precision: 0.5, recall: 0.0323
2019-01-17T

2019-01-17T20:46:40.468309, step: 179, loss: 0.7888889312744141, acc: 0.5625, auc: 0.6489, precision: 1.0, recall: 0.082
2019-01-17T20:46:41.205041, step: 180, loss: 0.6397580504417419, acc: 0.5938, auc: 0.7627, precision: 1.0, recall: 0.1475
2019-01-17T20:46:41.861815, step: 181, loss: 0.6206620335578918, acc: 0.625, auc: 0.6904, precision: 1.0, recall: 0.1724
2019-01-17T20:46:42.506069, step: 182, loss: 0.6419763565063477, acc: 0.6172, auc: 0.661, precision: 0.75, recall: 0.4348
2019-01-17T20:46:43.175642, step: 183, loss: 0.6942650675773621, acc: 0.6094, auc: 0.6188, precision: 0.6, recall: 0.619
2019-01-17T20:46:43.873139, step: 184, loss: 0.6516944169998169, acc: 0.6719, auc: 0.7235, precision: 0.6711, recall: 0.75
2019-01-17T20:46:44.535513, step: 185, loss: 0.627532958984375, acc: 0.6562, auc: 0.7415, precision: 0.6709, recall: 0.7465
2019-01-17T20:46:45.239200, step: 186, loss: 0.6456259489059448, acc: 0.6953, auc: 0.7483, precision: 0.7021, recall: 0.569
2019-01-17T20:46:45.98

2019-01-17T20:47:54.756564, step: 245, loss: 0.6842005848884583, acc: 0.4609, auc: 0.5851, precision: 0.0, recall: 0.0
2019-01-17T20:47:55.415266, step: 246, loss: 0.692381739616394, acc: 0.4844, auc: 0.526, precision: 0.0, recall: 0.0
2019-01-17T20:47:56.102974, step: 247, loss: 0.6576493978500366, acc: 0.6016, auc: 0.75, precision: 1.0, recall: 0.0893
2019-01-17T20:47:56.763933, step: 248, loss: 0.6651912331581116, acc: 0.5391, auc: 0.7328, precision: 1.0, recall: 0.0328
2019-01-17T20:47:57.469703, step: 249, loss: 0.6721345782279968, acc: 0.5703, auc: 0.5994, precision: 1.0, recall: 0.0678
2019-01-17T20:47:58.181584, step: 250, loss: 0.6672579050064087, acc: 0.5391, auc: 0.665, precision: 1.0, recall: 0.0328
2019-01-17T20:47:58.966400, step: 251, loss: 0.6653781533241272, acc: 0.5859, auc: 0.6518, precision: 1.0, recall: 0.0364
2019-01-17T20:47:59.643636, step: 252, loss: 0.6817029714584351, acc: 0.5078, auc: 0.5562, precision: 1.0, recall: 0.0156
2019-01-17T20:48:00.359417, step: 2

...

2019-01-17T20:54:18.453291, step: 634, loss: 0.29718077182769775, acc: 0.8906, auc: 0.9455, precision: 0.9194, recall: 0.8636
2019-01-17T20:54:19.123155, step: 635, loss: 0.2007565200328827, acc: 0.9219, auc: 0.9804, precision: 0.9672, recall: 0.8806
2019-01-17T20:54:19.770999, step: 636, loss: 0.28110525012016296, acc: 0.8906, auc: 0.9504, precision: 0.9333, recall: 0.8485
2019-01-17T20:54:20.493803, step: 637, loss: 0.245966374874115, acc: 0.8828, auc: 0.9624, precision: 0.9265, recall: 0.863
2019-01-17T20:54:21.243852, step: 638, loss: 0.26521188020706177, acc: 0.8984, auc: 0.9613, precision: 0.9138, recall: 0.8689
2019-01-17T20:54:21.917438, step: 639, loss: 0.20950017869472504, acc: 0.9219, auc: 0.9759, precision: 0.9455, recall: 0.8814
2019-01-17T20:54:22.568143, step: 640, loss: 0.18448716402053833, acc: 0.9219, auc: 0.9853, precision: 0.9661, recall: 0.8769
2019-01-17T20:54:23.253809, step: 641, loss: 0.31424790620803833, acc: 0.8828, auc: 0.9385, precision: 0.9444, recall: 0.8

2019-01-17T20:55:05.131755, step: 700, loss: 0.25209516286849976, acc: 0.9141, auc: 0.969, precision: 0.8824, recall: 0.9524

Evaluation:
2019-01-17T20:55:32.805456, step: 700, loss: 0.4021969078442989, acc: 0.8497564102564101, auc: 0.9103769230769233, precision: 0.8517743589743588, recall: 0.8522000000000001
Saved model checkpoint to ../model/Bi-LSTM/model/my-model-700

2019-01-17T20:55:34.215735, step: 701, loss: 0.19632109999656677, acc: 0.9375, auc: 0.9795, precision: 0.9153, recall: 0.9474
2019-01-17T20:55:34.907890, step: 702, loss: 0.3070557713508606, acc: 0.8906, auc: 0.9377, precision: 0.9048, recall: 0.8769
2019-01-17T20:55:35.582039, step: 703, loss: 0.13140656054019928, acc: 0.9609, auc: 0.9801, precision: 0.9808, recall: 0.9273
2019-01-17T20:55:36.317874, step: 704, loss: 0.26929986476898193, acc: 0.9062, auc: 0.9554, precision: 0.94, recall: 0.8393
2019-01-17T20:55:37.049681, step: 705, loss: 0.19361257553100586, acc: 0.9297, auc: 0.9597, precision: 0.9818, recall: 0.871


2019-01-17T20:56:18.497394, step: 764, loss: 0.22246429324150085, acc: 0.9141, auc: 0.9653, precision: 0.9219, recall: 0.9077
2019-01-17T20:56:19.131026, step: 765, loss: 0.25175368785858154, acc: 0.9219, auc: 0.9636, precision: 0.9365, recall: 0.9077
2019-01-17T20:56:19.790312, step: 766, loss: 0.3127237856388092, acc: 0.8672, auc: 0.9446, precision: 0.9286, recall: 0.8
2019-01-17T20:56:20.518600, step: 767, loss: 0.34583261609077454, acc: 0.875, auc: 0.9385, precision: 0.9623, recall: 0.7846
2019-01-17T20:56:21.200548, step: 768, loss: 0.26411736011505127, acc: 0.8594, auc: 0.96, precision: 0.8333, recall: 0.8333
2019-01-17T20:56:21.860932, step: 769, loss: 0.298117458820343, acc: 0.8438, auc: 0.9559, precision: 0.95, recall: 0.6786
2019-01-17T20:56:22.603531, step: 770, loss: 0.250280499458313, acc: 0.9219, auc: 0.9536, precision: 0.9492, recall: 0.8889
2019-01-17T20:56:23.311053, step: 771, loss: 0.2784368693828583, acc: 0.875, auc: 0.9594, precision: 0.8387, recall: 0.8966
2019-01

2019-01-17T20:57:31.667373, step: 828, loss: 0.23583680391311646, acc: 0.9141, auc: 0.969, precision: 0.8772, recall: 0.9259
2019-01-17T20:57:32.365172, step: 829, loss: 0.12794309854507446, acc: 0.9531, auc: 0.9892, precision: 0.9688, recall: 0.9394
2019-01-17T20:57:33.169616, step: 830, loss: 0.28193625807762146, acc: 0.9062, auc: 0.9389, precision: 0.931, recall: 0.871
2019-01-17T20:57:33.870559, step: 831, loss: 0.09028791636228561, acc: 0.9844, auc: 0.9908, precision: 0.9863, recall: 0.9863
2019-01-17T20:57:34.575366, step: 832, loss: 0.21294032037258148, acc: 0.9375, auc: 0.9604, precision: 0.9492, recall: 0.918
2019-01-17T20:57:35.312094, step: 833, loss: 0.19665855169296265, acc: 0.9375, auc: 0.9817, precision: 0.9254, recall: 0.9538
2019-01-17T20:57:36.060038, step: 834, loss: 0.20019574463367462, acc: 0.9375, auc: 0.9676, precision: 0.9577, recall: 0.9315
2019-01-17T20:57:36.788910, step: 835, loss: 0.20649504661560059, acc: 0.9453, auc: 0.9699, precision: 0.9231, recall: 0.9

KeyboardInterrupt: 